In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import matplotlib.pyplot as plt
os.chdir(r'C:\Users\38030\OneDrive\CORNELL\2020F\AB')

In [31]:
data=pd.read_csv('test_prc.csv')

In [32]:
data.sort_values(by=['tic','datadate'],inplace=True)
data.index=data.datadate.apply(lambda x: dt.datetime.strptime(str(x),'%Y%m%d'))
def ret(x):
    return(x/x.shift(1)-1)
data['ret']=data.groupby(['tic'])['prccm'].apply(ret)
data['trfm'].fillna(0,inplace=True)
data['adj_ret']=data['ret']*(data['trfm']/100+1)

In [37]:
count=data.groupby(['tic'])['adj_ret'].count().reset_index()
count_0=data[data.adj_ret==0].groupby(['tic'])['adj_ret'].count().reset_index()
count_0.columns=['tic','constant_prc']
count=pd.merge(count,count_0)
count['constant_ratio']=count['constant_prc']/count['adj_ret']

In [41]:
missing=count[count.constant_ratio>=0.15]
ticker=data[['tic','gvkey','conm','exchg']].drop_duplicates()
missing=pd.merge(ticker,missing)

In [42]:
missing1=count[count.adj_ret<=25]
missing1=pd.merge(ticker,missing1)
missing=missing.append(missing1)
missing.drop_duplicates(inplace=True)
missing.to_csv('compustat_missing.csv',index=False)

In [74]:
error=data[data.adj_ret>4]
temp=data[data.adj_ret<-0.75]
error=error.append(temp)
error.to_csv('large_return.csv',index=False)

In [46]:
missing

,tic,gvkey,conm,exchg,adj_ret,constant_prc,constant_ratio
0,4749B,7938,NACCO INDUSTRIES -CL A,1,32,32,1.000000
1,6525B,10793,TYSON FOODS INC -CL A,1,1,1,1.000000
2,6712B,66708,BROADCOM CORP,1,4,1,0.250000
3,AVATQ,7745,AVATEX CORP,19,28,14,0.500000
4,BNSSA,2433,BNS HOLDING CO,19,29,10,0.344828
5,HTMXQ,5505,HARTMARX CORP,19,40,7,0.175000
6,LTVCQ,6531,LTV CORP,19,21,5,0.238095
7,MDPEB,7260,MEREDITH CORP,19,119,56,0.470588
8,NYTAB,7866,NEW YORK TIMES CO -CL A,19,85,85,1.000000
9,PALM,132502,PALM INC,14,6,1,0.166667


In [83]:
error

,gvkey,iid,datadate,tic,conm,prccm,trfm,exchg,ret,adj_ret
datadate,,,,,,,,,,
2011-07-31,7745,01,20110731,AVATQ,AVATEX CORP,0.0100,4.0391,19,32.333333,33.639309
2011-08-31,7745,01,20110831,AVATQ,AVATEX CORP,0.4490,4.0391,19,43.900000,45.673165
2011-11-30,7745,01,20111130,AVATQ,AVATEX CORP,0.0095,4.0391,19,14.833333,15.432467
2010-08-31,2433,01,20100831,BNSSA,BNS HOLDING CO,221.0000,6.7758,19,18.644444,19.907755
2015-10-31,142460,01,20151031,BTUUQ,PEABODY ENERGY CORP,12.7900,1.2284,19,8.268116,8.369681
...,...,...,...,...,...,...,...,...,...,...
2012-03-31,16243,01,20120331,WAMUQ,WASHINGTON MUTUAL INC,0.0010,1.9681,19,-0.972678,-0.991821
2019-05-31,4367,01,20190531,WFTIF,WEATHERFORD INTL PLC,0.0465,2.0835,19,-0.915959,-0.935043
2019-02-28,174490,01,20190228,WINMQ,WINDSTREAM HOLDINGS INC,0.4000,11.0399,19,-0.867110,-0.962838
